In [ ]:
#@title [RUN] Setup Program and Prepare CAT & DOG Dataset

import torch
import torch.nn.functional as func
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np

# ตั้งค่าอุปกรณ์ (GPU/CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# ตั้งค่าขนาดภาพและ Batch Size
image_width = image_height = 224
batch_size = 64

# จำนวนคลาส
class_number = 2
class_labels = ["cat", "dog"]

# Image Transform
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# โหลด Dataset
dataset = datasets.ImageFolder(root="catdog_dataset/train", transform=transform)

# แบ่ง Dataset เป็น Train (80%) และ Validation (20%)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# สร้าง DataLoader สำหรับ Train และ Validation
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

# โหลด Test Dataset
test_dataset = datasets.ImageFolder(root="catdog_dataset/test", transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# แสดงข้อมูลเกี่ยวกับ Dataset
print(f"📌 Train Samples: {len(train_dataset)}, Validation Samples: {len(val_dataset)}, Test Samples: {len(test_dataset)}")

# ฟังก์ชันแสดงภาพจาก DataLoader
def show_images_grid(dataloader, row=8):
    images, labels = next(iter(dataloader))
    images = images[:(row*row)]
    grid = torchvision.utils.make_grid(images, nrow=row, normalize=True, padding=2)
    np_grid = grid.permute(1, 2, 0).numpy()
    plt.figure(figsize=(10, 10))
    plt.imshow(np_grid)
    plt.axis('off')
    plt.title(f"Sample Images from DataLoader (Batch size: {len(images)})")
    plt.show()

# แสดงตัวอย่างภาพจาก Validation Set
show_images_grid(val_loader)

In [ ]:
import os
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset

# ตั้งค่าอุปกรณ์
device = "cuda" if torch.cuda.is_available() else "cpu"

# ตั้งค่าขนาดภาพและ batch size
image_size = 224
batch_size = 64

# คลาสที่เราจะใช้จาก Tiny ImageNet (เฉพาะหมาและแมว)
cat_dog_classes = ["n02124075", "n02123159", "n02123394", "n02123597", "n02085620", "n02085782"]

# Image Transform
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

# โหลด Dataset
dataset = datasets.ImageFolder(root="tiny-imagenet-200/train", transform=transform)

# กรองเฉพาะคลาสหมาและแมว
filtered_indices = [i for i, (img, label) in enumerate(dataset) if dataset.classes[label] in cat_dog_classes]
filtered_dataset = Subset(dataset, filtered_indices)

# แบ่งเป็น Train (80%) และ Validation (20%)
train_size = int(0.8 * len(filtered_dataset))
val_size = len(filtered_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(filtered_dataset, [train_size, val_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# แสดงข้อมูลเกี่ยวกับ Dataset
print(f"📌 Train Samples: {len(train_dataset)}, Validation Samples: {len(val_dataset)}")

# ทดสอบแสดงตัวอย่างภาพ
import matplotlib.pyplot as plt
import torchvision

def show_images_grid(dataloader, row=8):
    images, labels = next(iter(dataloader))
    images = images[:(row*row)]
    grid = torchvision.utils.make_grid(images, nrow=row, normalize=True, padding=2)
    np_grid = grid.permute(1, 2, 0).numpy()
    plt.figure(figsize=(10, 10))
    plt.imshow(np_grid)
    plt.axis('off')
    plt.title(f"Sample Images (Batch size: {len(images)})")
    plt.show()

# แสดงตัวอย่างจาก Train Set
show_images_grid(train_loader)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import vit_b_16, ViT_B_16_Weights
import numpy as np

# ใช้ GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CutMix
def rand_bbox(size, lam):
    W, H = size[2], size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx, cy = np.random.randint(W), np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)
    
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bbx1:bbx2, bby1:bby2] = x[index, :, bbx1:bbx2, bby1:bby2]
    
    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam

# Data Augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomErasing(p=0.3),
])

# โหลดโมเดล ViT พร้อม Pretrained Weights
weights = ViT_B_16_Weights.IMAGENET1K_V1
model = vit_b_16(weights=weights)
model.heads.head = nn.Sequential(
    nn.LayerNorm(model.hidden_dim),
    nn.Dropout(0.3),  # ป้องกัน Overfitting
    nn.Linear(model.hidden_dim, 2)
)
model = model.to(device)

# ใช้ Mixed Precision (fp16)
scaler = torch.cuda.amp.GradScaler()

# Hyperparameters
learning_rate = 1e-4
num_epochs = 50

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# Loss Function
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Learning Rate Scheduler (Cosine Annealing)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

# SWA (Stochastic Weight Averaging)
from torch.optim.swa_utils import AveragedModel, SWALR

swa_model = AveragedModel(model)
swa_scheduler = SWALR(optimizer, anneal_strategy="cos", swa_lr=1e-5)

# Test the model with fp16
input_tensor = torch.randn(1, 3, 224, 224).to(device)

with torch.cuda.amp.autocast():  # ใช้ fp16
    output = model(input_tensor)

print(output)

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

losses = []
val_losses = []
target_loss = 0.0010  # ค่า loss ที่ต้องการให้หยุดเทรน
patience = 200  # จำนวน epoch ที่รอ ถ้า Validation Loss ไม่ลด
best_val_loss = np.inf
wait = 0  # ตัวแปรเก็บจำนวน epoch ที่ Validation Loss ไม่ลดลง

# Training loop
for epoch in range(num_epochs):
    print(f"Epoch[{epoch + 1}/{num_epochs}] training ", end=" ")
    model.train()  # ตั้งค่าเป็น Train Mode

    epoch_loss = 0  # เก็บค่า loss ของ epoch นี้
    for data, targets in train_loader:
        # Move data and targets to the device (GPU/CPU)
        data = data.to(device)
        targets = targets.to(device)

        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()

        # Gradient Clipping (ป้องกัน Gradient Explosion)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        # Tracking losses
        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)  # คำนวณ loss เฉลี่ยต่อ epoch
    losses.append(epoch_loss)
    print('Train Loss: {:.4f}'.format(epoch_loss))

    # ================== Validation Phase ==================
    model.eval()  # ตั้งค่าเป็น Eval Mode (ปิด Dropout/BatchNorm)
    val_loss = 0
    with torch.no_grad():  # ปิดการคำนวณ Gradient
        for val_data, val_targets in val_loader:
            val_data = val_data.to(device)
            val_targets = val_targets.to(device)

            val_scores = model(val_data)
            val_loss += criterion(val_scores, val_targets).item()

    val_loss /= len(val_loader)  # คำนวณค่าเฉลี่ยของ Validation Loss
    val_losses.append(val_loss)
    print('Validation Loss: {:.4f}'.format(val_loss))

    # เช็คว่าค่า loss ต่ำกว่าที่กำหนดหรือยัง
    if epoch_loss < target_loss:
        print(f"✅ Loss ต่ำกว่า {target_loss:.4f}, หยุดการเทรน 🚀")
        break  # หยุดการเทรน

    # Early Stopping ถ้า Validation Loss ไม่ลดลง
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        wait = 0  # Reset ค่า wait
    else:
        wait += 1
        if wait >= patience:
            print(f"⏹ Early stopping! Validation Loss ไม่ลดลง {patience} epoch ติดต่อกัน")
            break

# Plot Training & Validation Loss
plt.plot(losses, label="Train Loss", marker="o")
plt.plot(val_losses, label="Validation Loss", marker="s")
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
#@title [RUN] Report Results: เซ็คความแม่นยำโมเดล

import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

def check_accuracy(loader, model, text=''):
    num_correct = 0
    num_samples = 0
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"{text}Got {num_correct}/{num_samples} correct ({accuracy:.2f}%)")

    model.train()
    return accuracy


def show_images_with_predictions(dataset, model, row=4, device="cpu"):
    images, labels = zip(*[dataset[i] for i in range(row * row)])
    images = torch.stack(images).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(images)
    _, predicted = torch.max(outputs, 1)

    grid = torchvision.utils.make_grid(images, nrow=row, normalize=True, padding=2)
    np_grid = grid.permute(1, 2, 0).cpu().numpy()

    if np_grid.shape[-1] == 1:
        np_grid = np_grid.squeeze(-1)
        cmap = "gray"
    else:
        cmap = None

    plt.figure(figsize=(10, 10))
    plt.imshow(np_grid, cmap=cmap)
    plt.axis('off')
    plt.title("Sample Images from Dataset with Predictions")

    for i in range(row * row):
        x = (i % row) * (np_grid.shape[1] // row) + (np_grid.shape[1] // row) // 2
        y = (i // row) * (np_grid.shape[0] // row) + (np_grid.shape[0] // row) // 2
        color = "green" if predicted[i] == labels[i] else "red"
        plt.text(x, y, f"{class_labels[predicted[i].item()]} ({'✔' if predicted[i] == labels[i] else '✖'})",
                 color=color, ha='center', va='center', fontsize=8, fontweight='bold')

    plt.show()
    model.train()


# Run the accuracy check and show results
train_acc = check_accuracy(train_loader, model, "TRAIN dataset accuracy: ")
test_acc = check_accuracy(test_loader, model, "TEST dataset accuracy: ")



# Show sample images with predictions
show_images_with_predictions(test_dataset, model, 10, device=device)

In [ ]:
torch.save(model.state_dict(), "trained_model.pth")
torch.save(model, "full_model.pth")